In [2]:
import numpy as np
from typing import Optional, Tuple
from scipy.optimize import fsolve

In [ ]:
class fuselage:
    def __init__(self) -> None:
        self.length: Optional[float] = None
        self.diameter: Optional[float] = None

        self.tailboom_length: Optional[float] = None
        self.tailboom_diameter: Optional[float] = None

        self.hrztail_h: Optional[float] = None
        self.hrztail_b: Optional[float] = None
        self.hrztail_t: Optional[float] = None
    
    #downwash
    def Re_down(self) -> float:
        rho = 1.225
        V = 20
        # in the most simiplied way, take this as the lamba inflow for hover at that altitude
        Vin = 5
        velocity = V + Vin

        d = self.diameter
        nu = 1.2

        return (rho*velocity*d)/nu

    def Downwash(self) -> float:
        Re = self.Re_down()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        Ki = 1.3
        SA = (np.pi)*self.diameter*self.length

        return SA*Ki*cd

    # backwards drag
    def Re_back_body(self):
        rho = 1.225
        velocity = 20
        d = self.length
        nu = 1.2

        return (rho*velocity*d)/nu
    
    def D1(self) -> float:
        Re = self.Re_back_body()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.diameter*self.length

        return SA*cd
    
    def Re_back_tail(self):
        rho = 1.225
        velocity = 20
        d = self.tailboom_length
        nu = 1.2

        return (rho*velocity*d)/nu
    
    def D2(self) -> float:
        Re = self.Re_back_tail()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.tailboom_diameter*self.length

        return SA*cd
    
    def Re_back_hrztail(self):
        rho = 1.225
        velocity = 20
        d = self.hrztail_b
        nu = 1.2

        return (rho*velocity*d)/nu
    
    def D3(self) -> float:
        Re = self.Re_back_hrztail()
        cd = (0.455/(np.log10(Re))) - (1700/Re)
        SA = (np.pi)*self.hrztail_b*self.length

        return SA*cd
    
    def output(self):
        return self.D1() + self.D2() + self.D3()

TRIM CONDITIONS:

In [ ]:
class Rotor:
    def __init__(self) -> None:
        self.Rroot: Optional[float] = None
        self.Rtip: Optional[float] = None
        
        self.thetaroot: Optional[float] = None
        self.thetatwist: Optional[float] = None

        self.Croot: Optional[float] = None
        self.Cslope: Optional[float] = None

        self.tailRroot: Optional[float] = None
        self.tailRtip: Optional[float] = None

        self.mass: Optional[float] = None

class flight_conditions:
    def __init__(self) -> None:
        self.velocity: Optional[float] = None
        self.rho: Optional[float] = None


class Trim:
    def __init__(self, rotor: Rotor, condition: flight_conditions) -> None:
        self.weight: Optional[float] = None
        self.velocity: Optional[float] = None
        self.rotor = rotor
        self.condition = condition
    
    def get_thrust(self, tail = 0) -> None:
        # get the drag
        drag = 20
        weight = self.weight

        return np.sqrt((weight)**2 + (drag)**2 + (tail)**2)
    
    def get_Vin(self, i = 0, Thrust: Optional[float] = None) -> float:
        guess = 2
        def thrust(Vin):
            rho = self.condition.rho
            V = self.condition.velocity
            A = np.pi*(self.Rtip)**2

            T = Thrust if Thrust is not None else self.get_thrust()
            tpp = self.alpha_tpp()

            return rho*A*(np.sqrt((V*np.cos(tpp)**2) + (V*np.sin(tpp) + Vin)**2))*2*Vin - T
        
        return fsolve(thrust, guess)[0]


    #Getting the beta0

    #alpha tpp  
    def alpha_tpp(self) -> None:
        drag = 0.5*60*self.condition.density*(self.velocity)**2
        return np.arctan(drag/self.weight)
    
    #pitch - theta
    def get_pitch(self, r,phi,t0: Optional[float] = None, t1c = 0, t1s = 0, b1c: Optional[float] = None, b1s = 0) -> float:
        t0 = t0 if t0 is not None else self.rotor.theta_root
        b1c = b1c if b1c is not None else self.alpha_tpp()
        c = t0 + (t1c - b1s)*np.cos(phi) + (b1c + t1s)*np.sin(phi)
        return c + r*self.rotor.thetatwist
    
    #get chord length
    def get_chord(self, r) -> float:
        return self.rotor.Croot+ r*self.rotor.Cslope
    
    #alpha
    def get_alpha(self, phi, r, t0, t1c, t1s, b1c, b1s, beta = 0) -> float:
        tpp = self.alpha_tpp()
        Vin = self.get_Vin()
        pitch = self.get_pitch(r, phi, t0, t1c, t1s, b1c, b1s)

        V = self.condition.velocity
        omega = (450*np.pi)/30

        advratio = V/omega*(self.rotor.Rtip - self.rotor.Rroot)
        lambag = Vin + V*np.sin(tpp)

        #vin
        vin = (Vin/omega*(self.rotor.Rtip - self.rotor.Rroot))*(1 + ((4*advratio)/(1.2*lambag + advratio)*3)*(r*np.cos(phi))/((self.rotor.Rtip - self.rotor.Rroot)))

        #iteration with beta
        P = V*np.sin(tpp) + vin + V*np.sin(beta)*np.cos(phi)
        T = omega*r + V*np.cos(tpp)*np.sin(phi)

        return np.deg2rad(pitch) - np.arctan(P/T)
    
    #cl function
    def get_cl(self, phi, r, beta0, t0, t1c, t1s, b1c, b1s):
        
        alpha = self.get_alpha(phi, r, t0, t1c, t1s, b1c, b1s, beta0)
        return 2*np.pi*alpha, alpha
    
    #cd function
    def get_cd(self, cl):
        return 0.01 + 0.2*cl**2
    
    def get_beta(self, t0, t1c, t1s, b1c, b1s) -> float:
        
        omega = (450*np.pi)/30
        beta0 = 0

        phi = np.deg2rad(45)
        I = (self.rotor.mass/3*(self.rotor.Rtip - self.rotor.Rroot))*((self.rotor.Rtip)**3 - (self.rotor.Rroot)**3)
        V = self.condition.velocity
    

        r = np.linspace(self.rotor.Rroot, self.rotor.Rtip, 100)
        dr = (self.rotor.Rtip - self.rotor.Rroot)/100
        

        while np.abs(beta - beta0) < 0.7:
            beta = beta0
            Vin = self.get_Vin()
            tpp = self.alpha_tpp()
            
            beta0 = 0
            for k in r:
                c = self.get_chord(r)
                cl = self.get_cl(self, phi, r, beta0, t0, t1c, t1s, b1c, b1s)[0]

                #assuming Vin doesnt change radially and azimuthally
                P = V*np.sin(tpp) + Vin + V*np.sin(beta)*np.cos(phi)
                T = omega*r + V*np.cos(tpp)*np.sin(phi)
                
                #sum is integration
                beta0 += np.sum(0.5*self.condition.rho*c*(P**2 + T**2)*r*cl*dr)
        
        return np.arcsin(beta0)

    
    def main_thrust_trim(self, Vin, beta0, t0, t1c, t1s, b1c, b1s):
        
        tpp = self.alpha_tpp()
        n = 100
        omega = (450*np.pi)/30
        r = np.linspace(self.rotor.Rroot, self.rotor.Rtip, n)
        dr = (self.rotor.Rtip - self.rotor.Rroot)/n
        phi = np.linspace(0, 90, 4)
        V = self.condition.velocity
        advratio = V/omega*(self.rotor.Rtip - self.rotor.Rroot)
        lambag = Vin + V*np.sin(tpp)
        vin = (Vin/omega*(self.rotor.Rtip - self.rotor.Rroot))*(1 + ((4*advratio)/(1.2*lambag + advratio)*3)*(r*np.cos(phi))/((self.rotor.Rtip - self.rotor.Rroot)))
        
        T = []
        D = []
        for i in phi:
            t_blade_phi = 0
            d_blade_phi = 0
            for j in r:
                for k in [i, i + 90, i +180, 1 + 270]:
                    P = V*np.sin(tpp) + vin + V*np.sin(beta0)*np.cos(phi)
                    T = omega*r + V*np.cos(tpp)*np.sin(phi)

                    if T > 0:
                        cl = self.get_cl(np.deg2rad(k), j, beta0, t0, t1c, t1s, b1c, b1s)[0]
                        cd = self.get_cd(cl)

                        alpha = self.get_cl(np.deg2rad(k), j, beta0)[1]
                        
                        t_blade_phi += (0.5*self.condition.rho*(P**2 + T**2)*(cl*np.cos(alpha) - cd*np.sin(alpha))*dr)

                        d_blade_phi += (j*0.5*self.condition.rho*(P**2 + T**2)*(cd*np.cos(alpha) + cl*np.sin(alpha))*dr)

                    
                    elif T < 0:
                        cl = 2*np.pi*self.get_alpha(np.deg2rad(k), j, beta0)
                        cd = 0.1 + 0.02*cl**2

                        t_blade_phi += (0.5*self.condition.rho*(P**2 + T**2)*(cl*np.cos(alpha) - cd*np.sin(alpha))*dr).npcos(beta0)

                        d_blade_phi += (j*0.5*self.condition.rho*(P**2 + T**2)*(cd*np.cos(alpha) + cl*np.sin(alpha))*dr)
            T.append(t_blade_phi)
            D.append(d_blade_phi)
        
        return np.average(np.array(T)), np.average(np.array(D))

    def iteration(self):
        # Initial guesses
        t1c, t1s, t0 = 0.2, 0.0, 5.0
        b1s, Ltail = 0.0, 12.0
        omega = 450 * (np.pi / 30)
        R = self.Rtip - self.Rroot

        #for force balance
        Fx = 20
        Fy = self.weight

        tol = 1e-3
        error_fx, error_fy = 1, 1

        Ttail_prev = 0
        while abs(error_fx) > tol or abs(error_fy) > tol:
            b1c = self.alpha_tpp()
            Vin = self.get_Vin()
            beta = self.get_beta(t0, t1c, t1s, b1c, b1s)
            Ttail = self.main_thrust_trim(Vin, beta)[1] / Ltail
            T = self.get_thrust(Ttail)
            b1s = np.arcsin(Ttail / T)

            fx = T * np.cos(b1s) * np.sin(self.alpha_tpp()) - Fx
            fy = T * np.cos(b1s) * np.cos(self.alpha_tpp()) - Fy

            if fx > tol:
                t1c += 0.05
            elif fx < -tol:
                t1c -= 0.04

            if fy > tol:
                t0 += 0.05
            elif fy < -tol:
                t0 -= 0.04
            
            if Ttail > Ttail_prev:
                t1s += 0.05
            elif Ttail < Ttail_prev:
                t1s -= 0.04

            Ttail_prev = Ttail
            error_fx, error_fy = fx, fy

        beta0 = beta
        Fz = T * np.sin(b1s)
        Fy = T * np.cos(b1s) * np.cos(self.alpha_tpp())
        Fx = T * np.cos(b1s) * np.sin(self.alpha_tpp())

        return t0, t1s, t1c, b1s, b1c, beta0, Fx, Fy, Fz

        
        


    


IndentationError: expected an indented block after function definition on line 120 (1558108891.py, line 121)